In [47]:
import pandas as pd
import json
import numpy as np
import re
import difflib as kwrd
pd.options.display.max_rows = 500

In [48]:
def similar(org,name):
    org = [y.replace('-',' ').replace('(',' ').replace(')',' ') for y in list(org['name'])]
    name = name.replace('-',' ').replace('(',' ').replace(')',' ')
    match = kwrd.get_close_matches(name,org,n=3, cutoff=0.5) 
    if len(match) > 0:
        return match[0]
    return None

In [49]:
with open ('../backend/dev/resources/files/organisations.json', 'r') as orgs:
    old = json.loads(orgs.read())

In [50]:
old = pd.DataFrame(old)
old['abbr'] = old['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
old['name'] = old['name'].apply(lambda x: x.strip())
old['lname'] = old['name'].apply(lambda x: x.split('(')[0].lower())

In [51]:
new = pd.read_csv('./new_organisation.csv')
new['abbr'] = new['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
new['name'] = new['name'].apply(lambda x: x.strip())
new['lname'] = new['name'].apply(lambda x: x.split('(')[0].lower())
new['review_status'] = 'APPROVED'

In [52]:
old['match'] = old['name'].apply(lambda x: similar(new, x))

In [53]:
URL_REGEX = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
def getUrl(x):
    url = re.findall(URL_REGEX, x)
    if len(url) > 0:
        return "https://" + url[0][0].replace("https://","").replace("http://","")
    return None

In [54]:
merged = pd.merge(new, old, on="lname", how="outer", validate="m:m").replace([np.nan], [None])
merged['name'] = merged.apply(lambda x: x['name_x'] if x['name_x'] else x['name_y'], axis=1)
merged = merged[['id','name','country','country_group','type','url', 'program','contribution','expertise','review_status']]
merged['url'] = merged['url'].apply(lambda x: getUrl(x) if x else None)
merged = merged[['id','name','country','country_group','type','url', 'program','contribution','expertise','review_status']].sort_values('name')
merged['review_status'] = merged['review_status'].fillna("SUBMITTED")
merged['country_group'] = merged['country_group'].apply(lambda x: None if x == ' ' else x)

In [55]:
new_inserted = merged[merged['id'] != merged['id']].reset_index()
new_inserted['id'] = new_inserted.index + 1000

In [56]:
merged = pd.concat([merged[merged['id'] == merged['id']].sort_values('id'), new_inserted])

In [61]:
merged.to_csv('./result/organisation_merged.csv', index=False)

In [62]:
unlisted = merged[merged['review_status'] == 'SUBMITTED']
unlisted.to_csv('./result/organisation_unlisted.csv')

In [63]:
merged.to_json('../backend/dev/resources/files/organisations_new.json', orient='records')